# Labelling a NER dataset for retraining with SpanMarker

In [1]:
import argilla as rg
from datasets import load_dataset
from span_marker import SpanMarkerModel 

In [2]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
# Replace workspace with the name of your workspace
rg.init(
    api_url="https://ignacioct-argilla.hf.space",
    api_key="owner.apikey",
    workspace="admin"
)

In [3]:
model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-xlm-roberta-base-fewnerd-fine-super")
dataset = load_dataset("tomaarsen/conll2002", "es")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 8323
    })
    validation: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1915
    })
    test: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1517
    })
})

In [4]:
for sample in dataset["train"].select(range(5)):
    print(sample)

{'id': '0', 'document_id': 0, 'sentence_id': 0, 'tokens': ['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.'], 'pos_tags': [29, 21, 29, 22, 13, 59, 28, 21, 28, 22, 20], 'ner_tags': [5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]}
{'id': '1', 'document_id': 0, 'sentence_id': 1, 'tokens': ['-'], 'pos_tags': [16], 'ner_tags': [0]}
{'id': '2', 'document_id': 0, 'sentence_id': 2, 'tokens': ['El', 'Abogado', 'General', 'del', 'Estado', ',', 'Daryl', 'Williams', ',', 'subrayó', 'hoy', 'la', 'necesidad', 'de', 'tomar', 'medidas', 'para', 'proteger', 'al', 'sistema', 'judicial', 'australiano', 'frente', 'a', 'una', 'página', 'de', 'internet', 'que', 'imposibilita', 'el', 'cumplimiento', 'de', 'los', 'principios', 'básicos', 'de', 'la', 'Ley', '.'], 'pos_tags': [4, 28, 1, 40, 28, 13, 47, 28, 13, 47, 38, 4, 28, 40, 49, 28, 40, 49, 40, 28, 1, 1, 38, 40, 7, 28, 40, 28, 35, 47, 4, 28, 40, 4, 28, 1, 40, 4, 28, 20], 'ner_tags': [0, 1, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
labels = dataset["train"].features["ner_tags"].feature.names
print(labels)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [6]:
fine_labels = dataset["train"].features["pos_tags"].feature.names
print(fine_labels)

['AO', 'AQ', 'CC', 'CS', 'DA', 'DE', 'DD', 'DI', 'DN', 'DP', 'DT', 'Faa', 'Fat', 'Fc', 'Fd', 'Fe', 'Fg', 'Fh', 'Fia', 'Fit', 'Fp', 'Fpa', 'Fpt', 'Fs', 'Ft', 'Fx', 'Fz', 'I', 'NC', 'NP', 'P0', 'PD', 'PI', 'PN', 'PP', 'PR', 'PT', 'PX', 'RG', 'RN', 'SP', 'VAI', 'VAM', 'VAN', 'VAP', 'VAS', 'VMG', 'VMI', 'VMM', 'VMN', 'VMP', 'VMS', 'VSG', 'VSI', 'VSM', 'VSN', 'VSP', 'VSS', 'Y', 'Z']


In [7]:
for sample in dataset["train"].select(range(5)):
    print(model.predict(sample['tokens']))

/Users/ignacio/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/Users/ignacio/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[{'span': ['Melbourne'], 'label': 'location-GPE', 'score': 0.907048225402832, 'word_start_index': 0, 'word_end_index': 1}, {'span': ['Australia'], 'label': 'location-GPE', 'score': 0.9773393273353577, 'word_start_index': 2, 'word_end_index': 3}]
[]
[{'span': ['Daryl', 'Williams'], 'label': 'person-other', 'score': 0.7660976648330688, 'word_start_index': 6, 'word_end_index': 8}]
[{'span': ['Victoria'], 'label': 'location-GPE', 'score': 0.6540813446044922, 'word_start_index': 18, 'word_end_index': 19}, {'span': ['Australia'], 'label': 'location-GPE', 'score': 0.97244793176651, 'word_start_index': 20, 'word_end_index': 21}, {'span': ['CrimeNet'], 'label': 'product-software', 'score': 0.3033939301967621, 'word_start_index': 59, 'word_end_index': 60}]
[]


In [14]:
# Build records for the first 100 examples
records = []

for record in dataset["train"].select(range(5)):

    pred = model.predict(record['tokens'])

    print(pred)

    # Building TokenClassificationRecord
    records.append(
        rg.TokenClassificationRecord(
            text=" ".join(record["tokens"]),
            tokens=record["tokens"],
            prediction=pred,
            prediction_agent="tomaarsen/span-marker-xlm-roberta-base-fewnerd-fine-super",
        )
    )

# Log the records to Argilla
rg.log(records, name="conll2002_es", metadata={"split": "test"})

[{'span': ['Melbourne'], 'label': 'location-GPE', 'score': 0.907048225402832, 'word_start_index': 0, 'word_end_index': 1}, {'span': ['Australia'], 'label': 'location-GPE', 'score': 0.9773393273353577, 'word_start_index': 2, 'word_end_index': 3}]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/s0/4h4_2s7n3wxbm06wx8y43mm80000gn/T/ipykernel_62920/2993639455.py:12 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/s0/4h4_2s7n3wxbm06wx8y43mm80000gn/T/ipykernel_62920/2993639455.py'                 │
│                                                                                                  │
│ /Users/ignacio/Documents/recognai/argilla/src/argilla/client/models.py:429 in __init__           │
│                                                                                                  │
│   426 │   │   if text is None:                                                                   │
│   427 │   │   │   text = " ".join(tokens)                                                        │
│   428 │   │                                                                                      │
│ ❱ 429 │   │   super().__init__(text=text, tokens=tokens, **data)                                 │
│   430 │   │                                                                                      │
│   431 │   │   self._span_utils = SpanUtils(self.text, self.tokens)                               │
│   432                                                                                            │
│                                                                                                  │
│ /Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/feed │
│ back/pydantic/main.py:339 in pydantic.main.BaseModel.__init__                                    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/fee │
│ dback/pydantic/main.py'                                                                          │
│                                                                                                  │
│ /Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/feed │
│ back/pydantic/main.py:1076 in pydantic.main.validate_model                                       │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/fee │
│ dback/pydantic/main.py'                                                                          │
│                                                                                                  │
│ /Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/feed │
│ back/pydantic/fields.py:898 in pydantic.fields.ModelField.validate                               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/fee │
│ dback/pydantic/fields.py'                                                                        │
│                                                                                                  │
│ /Users/ignacio/Documents/recognai/argilla/docs/_source/tutorials_and_integrations/tutorials/feed │
│ back/pydantic/fields.py:1157 in pydantic.fields.ModelField._apply_validators                     │
│                                                            